In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from tqdm.notebook import tqdm
import random
import gc
import time

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
import lightgbm as lgb

In [ ]:
gc.enable()

One of the main problems that I faced is the large amount of data that needs to be handled, so I process the data one portion at a time in order to always keep under control the memory usage, applying similar transformations multiple times.

Application data

For all the csv files I consider XNA and XAP as nan (along with the default nan).

In [ ]:
train_data = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)
test_data = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

Cleaning the application data

In [ ]:
train_counts = train_data.count().sort_values()/len(train_data)
test_counts = test_data.count().sort_values()/len(test_data)

In [ ]:
cols = set(train_counts[(train_counts < 1) & (train_counts > 0.99)].index) - set(test_counts[(test_counts < 1) & (test_counts > 0.9)].index)

In [ ]:
cols

I drop a few rows in the train data where there are less than .01% missing values in columns where the test data has got no missing values and then I join the two datasets adding one column IS_TRAIN to identify where each record belongs.

In [ ]:
train_data.dropna(subset=cols, inplace=True)

In [ ]:
train_target = train_data[['SK_ID_CURR', 'TARGET']]

In [ ]:
submit = test_data[['SK_ID_CURR']]

In [ ]:
train_data.drop(columns=['TARGET'], inplace=True)

In [ ]:
test_data['IS_TRAIN'] = 0
train_data['IS_TRAIN'] = 1

In [ ]:
application_data = train_data.append(test_data)

In [ ]:
del(train_data)
del(test_data)

Analysis of columns with more than 60% of missing values

In [ ]:
appl_counts = application_data.count().sort_values()/len(application_data)

In [ ]:
appl_counts[(appl_counts < 0.6)]

OWN_CAR_AGE can be handled with FLAG_OWN_CAR when using the tree based gradient boosting.

EXT_SOURCE_1 will be imputed when the external bureau data is added.
The rest is data relative to the housing and it is mostly missing, so I drop all those columns.

In [ ]:
appl_counts = application_data.count().sort_values()/len(application_data)

In [ ]:
cols = list(set(appl_counts[(appl_counts < 0.6)].index) - set(['EXT_SOURCE_1', 'OWN_CAR_AGE']))

In the final submission I decided to also leave the housing features which slightly improve the score.

In [ ]:
#application_data.drop(columns=cols, inplace=True)

Label encoding for binary categorical features

In [ ]:
le = LabelEncoder()
for col in application_data.select_dtypes('object'):
    if len(application_data[col].unique()) <= 2:
        le.fit(application_data[col])
        application_data[col] = le.transform(application_data[col])

Next I handle the three categorical features with missing values: ORGANIZATION_TYPE, NAME_TYPE_SUITE, OCCUPATION_TYPE. The approach that I prefer to follow is to create a new categorical value for all of them called Nan in order to avoid messing up the existing data, which will be handled by the get_dummies function used for the one hot encoding. I will use the same approch for all the following data.

One hot encoding of all the other categorical features

In [ ]:
application_data = pd.get_dummies(application_data, dummy_na=True)

I check the correlation of the features with the target to see if I can drop the remaining columns with missing values

In [ ]:
appl_counts = application_data.count().sort_values()/len(application_data)

In [ ]:
appl_counts[(appl_counts < 1)]

In [ ]:
train_data = application_data[application_data.IS_TRAIN == 1].merge(train_target, how='left', on='SK_ID_CURR')

In [ ]:
corrs = train_data.corr()

In [ ]:
del(train_data)

In [ ]:
corrs['TARGET'].abs().sort_values().tail(40)

EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 are the best features, which are scores obtained from external sources. I tried to impute them using the rest of the data, but I only end up lowering the correlation with the target feature so I prefer to let them as they are since the the lgbm algorithm can handle missing data.

The 6 AMT_REQ_CREDIT_BUREAU features are missing because these clients are not present in the bureau credit data. So I add a new feature IS_IN_BUREAU.

In [ ]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

In [ ]:
application_data[application_data.AMT_REQ_CREDIT_BUREAU_WEEK.isnull() & application_data.SK_ID_CURR.isin(bureau.SK_ID_CURR.unique())]

In [ ]:
application_data['IS_IN_BUREAU'] = 0

In [ ]:
application_data.loc[application_data.SK_ID_CURR.isin(bureau.SK_ID_CURR.unique()), 'IS_IN_BUREAU'] = 1

As for the CNT_SOCIAL_CIRCLE features, not knowing how this data was gathered, I decided to add a feature HAS_SOCIAL_CIRCLE.

In [ ]:
appl_counts = application_data.count().sort_values()/len(application_data)
appl_counts[(appl_counts < 1)]

In [ ]:
application_data['HAS_SOCIAL_CIRCLE'] = 0

In [ ]:
application_data.loc[~application_data.OBS_30_CNT_SOCIAL_CIRCLE.isnull(), 'HAS_SOCIAL_CIRCLE'] = 1

Hand crafted features. The few application data features that actually regard the entity of the loan are only slightly correlated to the target. So I tried to build a few new features

In [ ]:
application_data

In [ ]:
application_data['AMT_CREDIT_FRAC'] = application_data.AMT_CREDIT / application_data.AMT_INCOME_TOTAL

In [ ]:
application_data['AMT_CREDIT_FRAC'] = application_data.AMT_ANNUITY / application_data.AMT_CREDIT

In [ ]:
application_data['AMT_GOODS_FRAC'] = application_data.AMT_GOODS_PRICE / application_data.AMT_CREDIT

In [ ]:
application_data['AMT_ANNUITY_FRAC'] = application_data.AMT_ANNUITY / application_data.AMT_INCOME_TOTAL

In [ ]:
application_data['AMT_DPD_DEF'] = application_data.DEF_30_CNT_SOCIAL_CIRCLE + application_data.OBS_30_CNT_SOCIAL_CIRCLE

Only AMT_GOODS_FRAC is actually a very good feature. I kept all the others leaving the machine learning algorithm to decide how to use them.

Bureau Balance data

In [ ]:
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

I drop the data of loans that are not related to clients in the application data.

In [ ]:
bureau = bureau[bureau.SK_ID_CURR.isin(application_data.SK_ID_CURR.unique())]

In [ ]:
bureau_balance = bureau_balance[bureau_balance.SK_ID_BUREAU.isin(bureau.SK_ID_BUREAU.unique())]

One-hot encoding

In [ ]:
bureau_balance = pd.get_dummies(bureau_balance)

Aggregation

In [ ]:
bureau_balance = bureau_balance.sort_values(['SK_ID_BUREAU', 'MONTHS_BALANCE'])

In [ ]:
temp = bureau_balance.groupby('SK_ID_BUREAU').size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
bureau_balance = bureau_balance.groupby('SK_ID_BUREAU').agg({'last', 'sum', 'mean'})

In [ ]:
bureau_balance.columns = bureau_balance.columns.map('_'.join)

In [ ]:
bureau_balance.reset_index(inplace=True)

In [ ]:
bureau_balance = bureau_balance.merge(temp, how='left', on='SK_ID_BUREAU')

In [ ]:
bureau_balance.columns = bureau_balance.columns.map(lambda x : 'BLN_' + x if x != 'SK_ID_BUREAU' else x)

Bureau data

In [ ]:
bureau = bureau.merge(bureau_balance, how='left', on='SK_ID_BUREAU')

In [ ]:
bureau.drop(columns='SK_ID_BUREAU', inplace=True)

In [ ]:
del(bureau_balance)

In [ ]:
bureau = bureau.sort_values(['SK_ID_CURR', 'DAYS_CREDIT'])

In [ ]:
bureau = pd.get_dummies(bureau, dummy_na=True)

In [ ]:
temp = bureau.groupby('SK_ID_CURR').size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
bureau = bureau.groupby('SK_ID_CURR').agg({'sum', 'mean', 'max'})

In [ ]:
bureau.columns = bureau.columns.map('_'.join)

In [ ]:
bureau.reset_index(inplace=True)

In [ ]:
bureau = bureau.merge(temp, how='left', on='SK_ID_CURR')

In [ ]:
bureau.columns = bureau.columns.map(lambda x : 'BRU_' + x if x != 'SK_ID_CURR' else x)

In [ ]:
application_data = application_data.merge(bureau, how='left', on='SK_ID_CURR')

In [ ]:
del(bureau)

Previous applications

In [ ]:
prev_application = pd.read_csv('/kaggle/input/home-credit-default-risk/previous_application.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

In [ ]:
prev_application = prev_application[prev_application.SK_ID_CURR.isin(application_data.SK_ID_CURR.unique())]

In [ ]:
prev_application = pd.get_dummies(prev_application, dummy_na=True)

In [ ]:
prev_application.drop(columns='SK_ID_PREV', inplace=True)

In [ ]:
prev_application = prev_application.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])

In [ ]:
temp = prev_application.groupby('SK_ID_CURR').size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
prev_application = prev_application.groupby('SK_ID_CURR').agg(['max', 'sum', 'mean']) # last

In [ ]:
prev_application.columns = prev_application.columns.map('_'.join)

In [ ]:
prev_application.reset_index(inplace=True)

In [ ]:
prev_application = prev_application.merge(temp, how='left', on='SK_ID_CURR')

In [ ]:
prev_application.columns = prev_application.columns.map(lambda x : 'PREV_' + x if x != 'SK_ID_CURR' else x)

In [ ]:
application_data = application_data.merge(prev_application, how='left', on='SK_ID_CURR')

In [ ]:
del(prev_application)

POS Cash Balance

In [ ]:
pos_cash_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

In [ ]:
pos_cash_balance = pos_cash_balance.sort_values(['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE'])

In [ ]:
temp = pos_cash_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV']).size().to_frame()
temp = temp.rename(columns={0: 'BLN_COUNT'})
temp.reset_index(inplace=True)

In [ ]:
pos_cash_balance = pd.get_dummies(pos_cash_balance, dummy_na=True)

In [ ]:
pos_cash_balance = pos_cash_balance.groupby(['SK_ID_PREV', 'SK_ID_CURR']).agg(['sum', 'mean', 'max']) # last

In [ ]:
pos_cash_balance.columns = pos_cash_balance.columns.map('_'.join)

In [ ]:
pos_cash_balance.reset_index(inplace=True)

In [ ]:
pos_cash_balance = pos_cash_balance.merge(temp, how='left', on=['SK_ID_CURR', 'SK_ID_PREV'])

In [ ]:
pos_cash_balance.drop(columns='SK_ID_PREV', inplace=True)

In [ ]:
temp = pos_cash_balance.groupby('SK_ID_CURR').size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
pos_cash_balance = pos_cash_balance.groupby(['SK_ID_CURR']).agg(['sum', 'mean', 'max']) 

In [ ]:
pos_cash_balance.columns = pos_cash_balance.columns.map('_'.join)

In [ ]:
pos_cash_balance.reset_index(inplace=True)

In [ ]:
pos_cash_balance = pos_cash_balance.merge(temp, how='left', on='SK_ID_CURR')

In [ ]:
pos_cash_balance.columns = pos_cash_balance.columns.map(lambda x : 'CSH_' + x if x != 'SK_ID_CURR' else x)

In [ ]:
application_data = application_data.merge(pos_cash_balance, how='left', on='SK_ID_CURR')

In [ ]:
del(pos_cash_balance)

Credit Card Balance

In [ ]:
credit_card_balance = pd.read_csv('/kaggle/input/home-credit-default-risk/credit_card_balance.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

In [ ]:
credit_card_balance = pd.get_dummies(credit_card_balance, dummy_na=True)

In [ ]:
credit_card_balance = credit_card_balance.sort_values(['SK_ID_CURR', 'SK_ID_PREV', 'MONTHS_BALANCE'])

In [ ]:
temp = credit_card_balance.groupby(['SK_ID_CURR', 'SK_ID_PREV']).size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
credit_card_balance = credit_card_balance.groupby(['SK_ID_PREV', 'SK_ID_CURR']).agg(['sum', 'mean', 'max']) # last

In [ ]:
credit_card_balance.columns = credit_card_balance.columns.map('_'.join)

In [ ]:
credit_card_balance.reset_index(inplace=True)

In [ ]:
credit_card_balance = credit_card_balance.merge(temp, how='left', on=['SK_ID_CURR', 'SK_ID_PREV'])

In [ ]:
credit_card_balance.drop(columns='SK_ID_PREV', inplace=True)

In [ ]:
credit_card_balance = credit_card_balance.groupby(['SK_ID_CURR']).agg(['sum', 'mean'])

In [ ]:
credit_card_balance.columns = credit_card_balance.columns.map('_'.join)

In [ ]:
credit_card_balance.reset_index(inplace=True)

In [ ]:
credit_card_balance.columns = credit_card_balance.columns.map(lambda x : 'CRD_' + x if x != 'SK_ID_CURR' else x)

In [ ]:
application_data = application_data.merge(credit_card_balance, how='left', on='SK_ID_CURR')

In [ ]:
del(credit_card_balance)

Installment payments

In [ ]:
installments_payments = pd.read_csv('/kaggle/input/home-credit-default-risk/installments_payments.csv', 
                               na_values=['XNA', 'XAP'], na_filter=True)

In [ ]:
installments_payments = installments_payments.sort_values(['SK_ID_CURR', 'SK_ID_PREV', 'DAYS_ENTRY_PAYMENT'])

In [ ]:
temp = installments_payments.groupby(['SK_ID_CURR', 'SK_ID_PREV']).size().to_frame()
temp = temp.rename(columns={0: 'COUNT'})
temp.reset_index(inplace=True)

In [ ]:
installments_payments.fillna(0, inplace=True)

In [ ]:
installments_payments = installments_payments.groupby(['SK_ID_PREV', 'SK_ID_CURR']).agg(['sum', 'mean', 'max', 'min'])

In [ ]:
installments_payments.columns = installments_payments.columns.map('_'.join)

In [ ]:
installments_payments.reset_index(inplace=True)

In [ ]:
installments_payments = installments_payments.merge(temp, how='left', on=['SK_ID_CURR', 'SK_ID_PREV'])

In [ ]:
installments_payments.drop(columns='SK_ID_PREV', inplace=True)

In [ ]:
installments_payments = installments_payments.groupby(['SK_ID_CURR']).agg(['sum', 'mean', 'max', 'min'])

In [ ]:
installments_payments.columns = installments_payments.columns.map('_'.join)

In [ ]:
installments_payments.reset_index(inplace=True)

In [ ]:
installments_payments.columns = installments_payments.columns.map(lambda x : 'INS_' + x if x != 'SK_ID_CURR' else x)

In [ ]:
application_data = application_data.merge(installments_payments, how='left', on='SK_ID_CURR')

In [ ]:
del(installments_payments)

In [ ]:
for col in application_data.columns:
    if len(application_data[col].unique()) <= 1:
        application_data.drop(columns=col,inplace=True)

In [ ]:
application_data.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in application_data.columns]

Feature selection

In order to reduce the number of features before starting the training and evaluation, I use the lgbm algorithm to select the most important features based on the number of times the feature is used in a model. 

In [ ]:
model = lgb.LGBMClassifier()

In [ ]:
train_data = application_data[application_data.IS_TRAIN == 1.0]

In [ ]:
test_data = application_data[application_data.IS_TRAIN == 0.0]

In [ ]:
train_data.drop(columns='IS_TRAIN', inplace=True)
test_data.drop(columns='IS_TRAIN', inplace=True)

In [ ]:
del(application_data)

In [ ]:
params = model.get_params()

In [ ]:
params['objective'] = 'binary'
params['metric'] = 'auc'

In [ ]:
skf = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
final_importance = np.zeros(len(train_data.columns))
for n_fold, (train_index, valid_index) in tqdm(enumerate(skf.split(train_data, train_target.TARGET))):
    X_train = train_data.iloc[train_index]
    y_train = train_target.iloc[train_index].TARGET
    X_valid = train_data.iloc[valid_index]
    y_valid = train_target.iloc[valid_index].TARGET
    lgb_train = lgb.Dataset(data=X_train, label=y_train)
    lgb_eval = lgb.Dataset(data=X_valid, label=y_valid)
    model = lgb.train(params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=150, verbose_eval=100)
    final_importance += model.feature_importance()

In [ ]:
fi = pd.DataFrame()
fi['FEAT'] = train_data.columns

In [ ]:
fi['importance'] = final_importance

In [ ]:
fi = fi.sort_values(by='importance', ascending=False)

In [ ]:
fi = fi[fi.importance != 0]

In [ ]:
fi.head(30)

In [ ]:
cols = list(set(fi.FEAT.values).union(set(['SK_ID_CURR'])))

In [ ]:
len(cols)

In [ ]:
train_data = train_data[cols]

In [ ]:
test_data = test_data[cols]

Hyperparameter Tuning

The approach that I follow for the hyperparameter tuning is the random search on the interval around the main default parameters of the lgbt classifier.

In [ ]:
def get_random_params():
    params = {
        'boosting_type': 'gbdt',
        'metric': 'auc',
        'num_leaves': random.randint(10, 60),
        'max_depth': random.randint(10, 30),
        'learning_rate': random.choice([0.0001, 0.0005, 0.001, 0.005, 0.01]),
        'n_estimators': random.randint(1000, 20000),
        'objective': 'binary',
        'reg_alpha': random.choice([0.001, 0.005, 0.01, 0.05, 0.1]),
        'reg_lambda': random.choice([0.001, 0.005, 0.01, 0.05, 0.1]),       
        'colsample_bytree': random.choice([0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
        'min_child_samples': random.randint(10, 100),
        'subsample_for_bin': random.randint(50000, 300000)
    }
    return params

In [ ]:
best_params = {'boosting_type': 'gbdt', 
               'metric': 'auc', 
               'num_leaves': 46, 
               'max_depth': 18, 
               'learning_rate': 0.01, 
               'n_estimators': 6289, 
               'objective': 'binary', 
               'reg_alpha': 0.05, 
               'reg_lambda': 0.05, 
               'colsample_bytree': 0.4, 
               'min_child_samples': 79, 
               'subsample_for_bin': 113092}
best_auc = 0.787228

In [ ]:
def get_best_params(hyper_rounds, n_folds, best_params=None, best_auc=0):
    best_params = best_params
    best_auc = best_auc
    lgb_train = lgb.Dataset(data=train_data, label=train_target.TARGET)
    for i in tqdm(range(hyper_rounds)):
        curr_params = get_random_params()
        start = time.time()
        print(curr_params)
        eval_hist = lgb.cv(curr_params, lgb_train, early_stopping_rounds = 200, nfold = n_folds, seed = 42, verbose_eval = 100)
        end = time.time()
        print('TIME:', end-start)
        curr_auc = eval_hist['auc-mean'][-1]
        if curr_auc > best_auc:
            best_params = curr_params
            best_auc = curr_auc
    return best_params, best_auc

remove the comment to do hyperparameter tuning

In [ ]:
HYPER_ROUNDS = 1
FOLDS = 5
#best_params, best_auc = get_best_params(HYPER_ROUNDS, FOLDS, best_params, best_auc)

Training and evaluation

In [ ]:
N_FOLDS = 10

In [ ]:
skf = StratifiedKFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
sub_preds = np.zeros(len(test_data))
avg_valid_auc = 0
for n_fold, (train_index, valid_index) in tqdm(enumerate(skf.split(train_data, train_target.TARGET))):
    print("FOLD N:", n_fold)
    X_train = train_data.iloc[train_index]
    y_train = train_target.iloc[train_index].TARGET
    X_valid = train_data.iloc[valid_index]
    y_valid = train_target.iloc[valid_index].TARGET
    lgb_train = lgb.Dataset(data=X_train, label=y_train)
    lgb_eval = lgb.Dataset(data=X_valid, label=y_valid)
    model = lgb.train(best_params, lgb_train, valid_sets=lgb_eval, early_stopping_rounds=150, verbose_eval=100)
    y_pred = model.predict(X_valid)
    sub_preds += model.predict(test_data) / skf.n_splits
    avg_valid_auc += roc_auc_score(y_valid, y_pred) / N_FOLDS

In [ ]:
avg_valid_auc

Submission

In [ ]:
submit['TARGET'] = sub_preds

In [ ]:
submit.to_csv('submission.csv', index = False)

References

1. Guolin Ke Qi Meng Thomas Finely Taifeng Wang Wei Chen Weidong Ma Qiwei Ye Tie-Yan Liu. 2017. [LightGBM: A Highly Efficient Gradient Boosting Decision Tree](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf)